In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import faiss
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [3]:
def load_data(data_path):
    loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [ ]:
#data_path = r"G:\End to End Medical Chatbot Using LLama2\data"
#extracted_data = load_data(data_path)
extracted_data=load_data("data/")

In [7]:
#extracted_data

In [9]:
# Create the  text chunks 

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    split_chunks=text_splitter.split_documents(extracted_data)
    return split_chunks

In [10]:
text_chunks=text_split(extracted_data)

In [11]:
len(text_chunks)

5699

In [12]:
# Load vector embedding model
def embedding_model():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embedding=embedding_model()

In [9]:
#Test Embedding model
query_res=embedding.embed_query("Hello world")
print(len(query_res))


384


In [10]:
#Load Vector store
vectordb=FAISS.from_texts([t.page_content for t in text_chunks],embedding)

In [11]:
vectordb

In [12]:
#Save the index and extract it 
vectordb.save_local("faiss_index")




In [13]:
vectordb = FAISS.load_local("faiss_index", embeddings=embedding)
query="What are Allergies"

docs=vectordb.similarity_search(query,k=3)
print("Result",docs)

Result [Document(page_content='Causes & symptoms\nOf all the causes of allergies , pollen is one of the\nmost widespread. Trees, weeds, and grasses produce\npollen in large amounts for seed production. These pol-\nlens are dispersed by the wind, and many never reach the\nintended targets. Instead, they are inhaled through the\nnose and throat. Different plants release their pollen at\ndifferent times of the year, so the timing of hay fever\nsymptoms varies from person to person, depending on\nwhich plants provoke a response.', metadata={}), Document(page_content='Allergen —A substance capable of producing an\nimmediate type of hypersensitivity , or allergy .\nWheal —A smooth, slightly elevated area on the\nbody surface, which is redder or paler than the\nsurrounding skin.\nGEAM - D to K  10/11/04 3:25 PM  Page 970', metadata={}), Document(page_content='harmless substances, the resulting reaction is called an\nallergy . An attack of hives is set off when such a sub-\nstance, called an a

In [17]:
#Create Prompt Template

prompt_template="""
Use the following information to answer the user's questions.
If you donot know any answer just say that i donot know, donot try to make up an answer.

Context:{context}
Question:{question}

Only return the helpful answer below and nothing else.
Helpful Answer:
"""

In [ ]:
PROMPT=PromptTemplate(
    template=prompt_template,
    input_variables=["context","question"]
)
chain_type_kwargs={"prompt": PROMPT}

In [14]:
from langchain.llms import CTransformers
llm=CTransformers(
    model="model\llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={"max_new_tokens": 512,
            "temperature":0.8}
)

ValidationError: 1 validation error for CTransformers
__root__
  Model path 'model\llama-2-7b-chat.ggmlv3.q4_0.bin' doesn't exist. (type=value_error)

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)